In [ ]:
!pip -q install git+https://github.com/mwshinn/PyDDM
import pyddm
import pyddm.plot
import matplotlib.pyplot as plt
import pandas
import numpy as np
from pyddm import Sample

In [ ]:
# Load data
df = pandas.read_csv('inference.csv')
df = df.dropna(subset=['RT'])

# tidy epoch data
transform_list = ['noise1frames_obs', 'noise2frames_obs', 'signal1frames_obs', 'signal2frames_obs',
                  'signal1Onset_design', 'noise2Onset_design', 'signal2Onset_design']
# df[transform_list] = df[transform_list].fillna(0.0001) / 72
df[transform_list] = df[transform_list] / 72

# make noise quartiles
# for col in transform_list[0:4]:
#     quartile_col = f"{col.split('_')[0]}_quartile"

#     # First create quartiles without labels
#     result = pd.qcut(df[col], q=4, duplicates='drop')

#     # Then convert the categories to simple 1-based integer labels
#     df[quartile_col] = result.cat.codes + 1

# split up 50% and 80% data
df['trueCue'] = df['trueCue'] / 100
df_50 = df[df['trueCue']==0.5]
df_80 = df[df['trueCue']==0.8]

# format the data for fitting
sample_50 = Sample.from_pandas_dataframe(df_50, rt_column_name="RT",
                                            choice_column_name="accuracy")

sample_80 = Sample.from_pandas_dataframe(df_80, rt_column_name="RT",
                                            choice_column_name="accuracy")

In [ ]:
# specify model for 80% cues
model_80 = pyddm.gddm(
    name = "starting point",
    starting_position = lambda bias, trueCongruence: bias if trueCongruence=='congruent' else -bias,
    bound="B",
    T_dur = 4.1,
    nondecision='ndt',
    parameters={'bias': (-1, 1), 'B': (0.1, 4), 'ndt': (0.2)},
    conditions = ['trueCongruence']
)

# visualize
# pyddm.plot.model_gui_jupyter(model_80, conditions={"trueCongruence": ['congruent', 'incongruent']}, sample=sample_80)
# pyddm.display_model(model_80)

# fit the model to 80% cue data
model_80.fit(sample=sample_80, verbose=False, lossfunction=pyddm.LossRobustBIC)
# pyddm.plot.plot_fit_diagnostics(model_80, sample=sample_80)
# model_80.parameters()

# specify the 50% model
model_50 = pyddm.gddm(
    name = "starting point",
    starting_position = 'bias',
    bound="B",
    T_dur = 4.1,
    nondecision='ndt',
    parameters={'bias': (-1, 1), 'B': (0.1, 4), 'ndt': (0.2)})

# visualize
# pyddm.plot.model_gui_jupyter(model_50, sample=sample_50)
# pyddm.display_model(model_50)

# fit the model to 50% cue data
model_50.fit(sample=sample_50, verbose=False)
pyddm.plot.plot_fit_diagnostics(model_50, sample=sample_50)
model_50.parameters()

In [ ]:
# manually estimate uncertainty around each parameter fit
model_80 = pyddm.gddm(
    name = "starting point",
    starting_position = lambda bias, trueCongruence: bias if trueCongruence=='congruent' else -bias,
    bound="B",
    T_dur = 4.1,
    nondecision='ndt',
    parameters={'bias': (-1, 1), 'B': (0.1, 4), 'ndt': (0.2)},
    conditions = ['trueCongruence']
)


# initialize bootstrapping
all_boundary_values = []
all_IC_values = []
n_bootstraps=100
data = df_80

# bootstrap
for i in range(n_bootstraps):

  # resample / shuffle data
  data.sample(frac=1, replace=True, random_state=1)

  # create new sample object (PyDDM expects conditions as a dictionary)
  bs_sample = Sample.from_pandas_dataframe(data, rt_column_name="RT", choice_column_name="accuracy")

  # iniatlize fitting model
  bs_model = model_80

  try:
    # fit model
    bs_model.fit(sample=bs_sample, verbose=False, lossfunction=pyddm.LossRobustBIC)

    # store parameter values
    bias = bs_model.parameters()['IC']
    all_IC_values.append(float(bias['bias']))
    boundary = bs_model.parameters()['bound']
    all_boundary_values.append(float(boundary['B']))

  except Exception as e:
    # fill in with NA on iterations where the model doesn't fit for whatever reason
    IC_values.append(np.nan)
    boundary_values.append(np.nan)
    print(f"Error in iteration {i}: {e}")

# compute number of unsuccessful iterations
np.count_nonzero(np.isnan(all_IC_values))

# # compute mean and std of non-nan value
mean_bias_80 = np.mean(all_IC_values)
std_bias_80 = np.std(all_IC_values)
mean_boundary_80 = np.mean(all_boundary_values)
std_boundary_80 = np.std(all_boundary_values)

plt.hist(all_IC_values, bins=25, color='purple', edgecolor='gray')
plt.xlabel('bias value')
plt.ylabel('frequency')
plt.title('distribution of starting point estimates for 80% trials')
plt.show()
print('mean = ', mean_bias_80, 'std = ', std_bias_80)

In [ ]:
# manually estimate uncertainty around each parameter fit
model_50 = pyddm.gddm(
    name = "starting point",
    starting_position = 'bias',
    bound="B",
    T_dur = 4.1,
    dt = 0.001,
    nondecision='ndt',
    parameters={'bias': (-1, 1), 'B': (0.1, 4), 'ndt': (0.2)})

# initialize bootstrapping
all_boundary_values_50 = []
all_IC_values_50 = []
n_bootstraps=100
data = df_50

# bootstrap
for i in range(n_bootstraps):

  # resample / shuffle data
  data.sample(frac=1, replace=True, random_state=1)

  # create new sample object (PyDDM expects conditions as a dictionary)
  bs_sample = Sample.from_pandas_dataframe(data, rt_column_name="RT", choice_column_name="accuracy")

  # iniatlize fitting model
  bs_model = model_50

  try:
    # fit model
    bs_model.fit(sample=bs_sample, verbose=False)

    # store parameter values
    bias = bs_model.parameters()['IC']
    all_IC_values_50.append(float(bias['x0']))
    boundary = bs_model.parameters()['bound']
    all_boundary_values_50.append(float(boundary['B']))

  except Exception as e:
    # fill in with NA on iterations where the model doesn't fit for whatever reason
    all_IC_values_50.append(np.nan)
    all_boundary_values_50.append(np.nan)
    print(f"Error in iteration {i}: {e}")

# compute number of unsuccessful iterations
np.count_nonzero(np.isnan(all_IC_values_50))

mean_bias_50 = np.mean(all_IC_values_50)
std_bias_50 = np.std(all_IC_values_50)

plt.hist(all_IC_values_50, bins=25, color='skyblue', edgecolor='gray')
plt.xlabel('bias value')
plt.ylabel('frequency')
plt.title('distribution of starting point estimates for 50% trials')
plt.show()
print('mean = ', mean_bias_50, 'std = ', std_bias_50)
print('mean = ', np.mean(all_boundary_values_50), 'std = ', np.std(all_boundary_values_50))

In [ ]:
plt.hist(all_IC_values, bins=25, color='purple', edgecolor='gray')
plt.hist(all_IC_values_50, bins=25, color='skyblue', edgecolor='gray')
plt.xlabel('bias value')
plt.ylabel('frequency')
plt.title('fitted starting points')
plt.legend(['80% trials', '50% trials'])
plt.show()


plt.hist(all_boundary_values, bins=25, color='purple', edgecolor='gray')
plt.hist(all_boundary_values_50, bins=25, color='skyblue', edgecolor='gray')
plt.xlabel('bound value')
plt.ylabel('frequency')
plt.title('fitted boundary values')
plt.legend(['80% trials', '50% trials'])
plt.show()